In [1]:
import os
# import nltk
# nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
import pickle
from bs4 import BeautifulSoup as bs
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords,wordnet
import json
import re
import operator

In [2]:
def sentence_break(reviews):
    sentences = []
    for text in reviews:
        p1 = re.compile("[\w)'][.?!]+(\s[A-Z0-9])*")
        p2 = re.compile("\d*\.\d+")
        while p2.search(text):
            a,b = p2.search(text).span()
#             print(text[a:b])
            text = text.replace(text[a:b],"0")
#             print(text[a:b])
        st = 1
        temp = 0
        end = 0
        for it in p1.finditer(text):
            end,temp = it.span() 
            if temp==len(text):
                sentences.append(text[st-1:end+1])
                break
            sentences.append(text[st-1:end+1])
            st = temp
    return sentences

In [3]:
def check_accuracy(actual_output, expected_output):
    accuracy = [0]*5
    count = [0]*5
    for i, a in enumerate(actual_output):
        e = expected_output[i]
        for j, x in enumerate(a):
            y = e[j]
            if y==3:
                accuracy[j]+=1
                # count[j]-=1
            elif x*y>0:
                accuracy[j]+=1
            elif x*y==0:
                count[j]-=1
            count[j]+=1

    accuracy = [float(v)/count[i] for i, v in enumerate(accuracy)]
    return accuracy

In [4]:
def sentences(text):
    sentence = []
    p1 = re.compile("[\w)'][.?!]+(\s[A-Z0-9])*")
    p2 = re.compile("\d*\.\d+")
    while p2.search(text):
        a,b = p2.search(text).span()
#         print(text[a:b])
        text = text.replace(text[a:b],"0")
#         print(text[a:b])
    st = 1
    temp = 0
    end = 0
    for it in p1.finditer(text):
        end,temp = it.span() 
        if temp==len(text):
            sentence.append(text[st-1:end+1])
            break
        sentence.append(text[st-1:end+1])
        st = temp
    return sentence

In [7]:
# assumes that data is in .json files in 'json' folder in the same folder as source code
# tested on TripAdvisor data in json format
data_path = os.path.join(os.path.dirname(os.getcwd()), 'json')
print(data_path)

all_reviews = []
all_ratings = []
count = 0
for filename in os.listdir(data_path):
    file_path = os.path.join(data_path, filename)
    fp = open(file_path, 'r')
    data = json.load(fp)['Reviews']
    for review in data:
#         print(review["Content"])
        all_reviews.append(sentences(review['Content']))
        all_ratings.append(review["Ratings"])
    count+=1
    if count>=10:
        break
print(all_ratings)
# print(all_reviews[0])

/home/sanjayr/Documents/nlp-project/absa/json
[{u'Service': u'5', u'Cleanliness': u'4', u'Overall': u'4.0', u'Value': u'4', u'Sleep Quality': u'5', u'Rooms': u'4', u'Location': u'4'}, {u'Service': u'5', u'Cleanliness': u'5', u'Overall': u'5.0', u'Value': u'5', u'Sleep Quality': u'5', u'Rooms': u'5', u'Location': u'5'}, {u'Service': u'5', u'Cleanliness': u'4', u'Overall': u'5.0', u'Value': u'5', u'Rooms': u'5', u'Location': u'5'}, {u'Service': u'4', u'Cleanliness': u'4', u'Overall': u'4.0', u'Value': u'5', u'Sleep Quality': u'4', u'Rooms': u'4', u'Location': u'4'}, {u'Service': u'3', u'Cleanliness': u'4', u'Overall': u'3.0', u'Value': u'4', u'Sleep Quality': u'4', u'Rooms': u'3', u'Location': u'4'}, {u'Service': u'5', u'Cleanliness': u'5', u'Overall': u'5.0', u'Value': u'5', u'Sleep Quality': u'5', u'Rooms': u'4', u'Location': u'4'}, {u'Service': u'5', u'Cleanliness': u'5', u'Overall': u'5.0', u'Value': u'5', u'Sleep Quality': u'4', u'Rooms': u'4', u'Location': u'5'}, {u'Service': u'4',

In [8]:
aspects = {"service":["staff",
                      "breakfast","restaurant",
                      "pool","food",
                      "desk","drink",
                      "tasty","tea",
                      "bar","dinner",
                      "lunch","delicious",
                      "spa","wi-fi",
                      "gym","gymnasium",
                      "internet","ample",
                      "parking","wireless",
                      "broken","polite",
                      "friendly","quick"],
           "cleanliness":["clean","dirty",
                          "messy","hygienic",
                          "smell",
                          "dry","wet",
                          "garbage","dusty"
                         "damp","clear",
                         "fresh","neat",
                          "clean","dirty",
                          "messy","hygienic",
                          "smell",
                          "dry","wet",
                          "garbage","dusty"
                         "damp","clear",
                         "fresh","neat",],
           "location":["place","trip",
                       "airport","railway",
                       "view","station",
                       "distance","far",
                       "close","train",
                       "metro","convenient",
                       "landmark","monument",
                       "temple","mosque",
                       "church","diner",
                       "mall","market",
                      "place","airport",
                      "station","mountains",
                      "mall","market","traffic"],
           "rooms":["bathroom","floor",
                    "door","stay",
                    "bed",
                    "shower","fridge",
                    "view","charge",
                   "window","electricity",
                   "bathroom","floor",
                    "door","stay",
                    "bed",
                    "shower","fridge",
                    "view","charge",
                   "window","electricity",],
           "value":["price","amount",
                    "rate","cheap",
                    "worth","low",
                    "money","economic",
                    "reasonable","fee",
                    "expensive","charge"
                   "price","amount",
                    "rate","cheap",
                    "worth","low",
                    "money","economic",
                    "reasonable","fee",
                    "expensive","charge"]}

In [9]:
tot = 0
for i in range(len(all_reviews)):
    tot += len(all_reviews[i])
    print(tot)

20
29
33
37
46
59
77
81
86
95
114
140
146
152
171
178
187
192
235
240
245
253
262
268
285
297
299
305
311
313
322
351
375
383
390
398
415
422
433
438
444
460
464
479
502
505
508
520
531
537
556
565
570
575
587
589
594
609
621
630
648
656
676
682
692
699
704
716
726
748
749
778
790
802
819
829
839
853
859
861
867
875
879
887
895
907
914
918
925
933
943
953
957
966
978
985
995
1003
1013
1023
1035
1043
1054
1065
1065
1082
1086
1092
1106
1111
1133
1138
1156
1161
1172
1188
1201
1212
1220
1226
1240
1244
1252
1253
1265
1269
1282
1292
1305
1317
1326
1333
1340
1353
1367
1383
1387
1406
1421
1432
1447
1460
1480
1486
1490
1496
1508
1516
1522
1523
1529
1566
1569
1604
1624
1653
1657
1671
1683
1690
1696
1702
1706
1727
1738
1769
1796
1805
1811
1819
1826
1843
1876
1879
1894
1905
1913
1919
1932
1952
2011
2025
2040
2052
2056
2066
2080
2086
2092
2101
2107
2110
2113
2120
2129
2133
2149
2162
2165
2172
2174
2179
2183
2189
2195
2199
2201
2206
2214
2222
2255
2263
2268
2272
2284
2294
2309
2320
2328
2338
2354
23

# Classification of sentences to corresponding aspects

In [10]:
# print(all_reviews[0])
# for each review
aspect_sentences = []
c=0
for review in all_reviews:
#     print(review)
    classified_sentences = dict()
    for x in aspects:
        classified_sentences[x] = []
    l = []
    # print(len(review))
# for each sentence in the review
    for sentence in review:
#         print(sentence)
        aspect_score = dict()
        for x in aspects:
            aspect_score[x] = 0
        words = sentence.split(" ")
#     for each word in the sentence
        for w in words:
#             print("WORD",w)
            p3 = re.compile("[)(\.,;:'-]")
            if p3.search(w):
                w = p3.sub(r"",w)
#             print("SYNSET",wordnet.synsets(w.lower()))
            word = wordnet.synsets(w.lower(),wordnet.NOUN)
#             print("NOUN SYNSET",word)
            if word==[]:
                continue
            keywords = [] # synset representation of keywords of different aspects
            for a in aspects.keys():
#                 print(a)
                count=0
                # for each keyword in aspect[a]
                for keyword in aspects[a]:
#                     print(keyword)
                    temp = wordnet.synsets(keyword,wordnet.NOUN)
                    if temp ==[]:
                        continue
                    keywords.append(wordnet.synset(temp[0].name()))
#                     print(word[0].name(),keyword,wordnet.synset(word[0].name()).lch_similarity(keywords[-1]))
                    aspect_score[a] += wordnet.synset(word[0].name()).lch_similarity(keywords[-1])
                    count+=1
                aspect_score[a]/=count
#             print(aspect_score)
        asp = max(aspect_score.items(),key=operator.itemgetter(1))[0]
#         print(sentence)
#         print("ASPECT",asp)
        classified_sentences[asp].append(sentence)
#         print(classified_sentences[max(aspect_score.items(),key=operator.itemgetter(1))[0]])
    aspect_sentences.append(classified_sentences)
    c+=1
    print(c)
#     print(classified_sentences)
#     print(aspect_score)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


KeyboardInterrupt: 

In [11]:
aspect_sentences[-25]

{'cleanliness': [u'The room did have an awful citrus odor that took some time to adjust to and the in-room microwave did not work',
  u'I did not try the pool, but it always appeared to be clean and frequently occupied'],
 'location': [u'I would recommend a nicer hotel, but the price is hard to beat'],
 'rooms': [],
 'service': [],
 'value': [u'This hotel had one of the lowest prices in the Phoenix area and it delivered just about what I expected',
  u'The hotel did not receive my internet booking, but honored my rate (twenty dollars below normal price)',
  u'I had no problems with safety and there was very little nightime noise',
  u'Certainly not the death trap that other reviewers make it out to be']}

# Aspect-Based sentiment classification

In [8]:
from nltk import pos_tag
predicted = []
# for each categorized review
for i in range(len(aspect_sentences)):
#     print(aspect_sentences[i])
    predicted_rating = dict()
    for x in aspects:
        predicted_rating[x] = 0
#     for each aspect of a review
        aspect_score = dict()
        for x in aspects:
            aspect_score[x] = [0,0]
    for a in aspects:
#         for each sentence of each aspect
        for sent in aspect_sentences[i][a]:
#             print(sent)
            score = [0,0]              # Sentiment score of each sentence
            words = pos_tag(word_tokenize(sent))
#             for each word in the sentence
            for items in words:
                if items[1] == 'JJ' or items[1] == 'JJR' or items[1] == 'JJS':
                    wadj = wordnet.synsets(items[0],wordnet.ADJ)
                    if wadj==[]:
                        continue
                    word = swn.senti_synset(wadj[0].name())
                    score[0] += word.pos_score()
                    score[1] += word.neg_score()
                if items[1] == 'NN' or items[1] == 'NNS' or items[1] == 'NNP' or items[1] == 'NNPS':
                    wnn = wordnet.synsets(items[0],wordnet.NOUN)
                    if wnn==[]:
                        continue
                    word = swn.senti_synset(wnn[0].name())
                    score[0] += word.pos_score()
                    score[1] += word.neg_score()
                if items[1] == 'RB' or items[1] == 'RBR' or items[1] == 'RBS':
                    wadv = wordnet.synsets(items[0],wordnet.ADV)
                    if wadv==[]:
                        continue
                    word = swn.senti_synset(wadv[0].name())
                    score[0] += word.pos_score()
                    score[1] += word.neg_score()
#                 print(score)
            aspect_score[a][0] += score[0]
            aspect_score[a][1] += score[1]
#             print(score)
            if "not" in word_tokenize(sent) or "never" in word_tokenize(sent):
                temp = aspect_score[a][0]
                aspect_score[a][0] = aspect_score[a][1]
                aspect_score[a][1] = temp
#             print(aspect_score)
        for a in aspects:
#             print(aspect_score[a][0],aspect_score[a][1])
            if aspect_score[a][0]>aspect_score[a][1]:
#                 print(a,aspect_score[a])
                predicted_rating[a] = 1
            elif aspect_score[a][0]<aspect_score[a][1]:
                predicted_rating[a] = -1
            else:
                predicted_rating[a] = 0
#         print(predicted_rating)
    predicted.append(list(predicted_rating.values()))
    print(i)
#     print(predicted)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
aspects1 = {
	'Service': 0,
	'Cleanliness': 1,
	'Value': 2,
	'Rooms': 3,
	'Location': 4
}
predicted

[[0, -1, -1, 1, 1],
 [0, 0, 0, 0, 1],
 [1, 1, 0, 1, 1],
 [1, 1, 0, 1, 1],
 [1, 0, 0, 0, -1],
 [0, 0, 1, 0, 1],
 [0, 1, -1, 0, 1],
 [0, 0, 1, -1, 0],
 [0, -1, 1, 1, 1],
 [0, 1, 1, 0, -1],
 [1, 1, -1, 0, 1],
 [1, 0, -1, -1, 1],
 [1, 0, 1, 1, 0],
 [0, 1, 1, 1, 1],
 [1, 1, 1, 1, 0],
 [1, -1, -1, 0, -1],
 [-1, 1, -1, 1, -1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1],
 [0, 1, 1, 0, 0],
 [0, 0, 1, 1, 0],
 [1, 0, 0, 0, 1],
 [0, 1, -1, 1, 1],
 [1, 0, 0, 1, 1],
 [0, 0, 1, 0, 1],
 [1, 0, 0, 1, 1],
 [1, 0, -1, 1, 1],
 [1, 0, 0, 0, 0],
 [0, -1, 1, -1, -1],
 [1, 0, 1, 0, 1],
 [0, 0, 0, 0, 1],
 [1, -1, 1, 1, 1],
 [1, 0, 0, 0, 0],
 [1, 0, -1, 1, 0],
 [0, 0, 1, 1, -1],
 [0, -1, 1, 0, 1],
 [0, 0, 1, -1, 1],
 [1, 1, 1, 1, 1],
 [1, 0, 0, 1, -1],
 [-1, 0, 0, 0, 1],
 [0, 0, 1, 0, 1],
 [0, 0, 1, 1, 1],
 [0, 1, -1, 0, 0],
 [1, 1, -1, 0, 0],
 [1, 1, 1, 1, 1],
 [0, 0, -1, 1, -1],
 [1, 1, 0, 1, 1],
 [0, 0, 0, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 0, 1, 1, 1],
 [1, 1, 1, -1, 1],
 [-1, 0, 1, -1, 0],
 [1, -1, 1, -1, 0],
 [1, 0, 0

In [10]:
output = []
for ratings in all_ratings:
    b = [0]*5
    print(ratings)
    for aspect in aspects1:
#         aspect = aspect.tolower()
        try:
            r = int(ratings[aspect])
            b[aspects1[aspect]] = r - 3
            if r == 3:
                b[aspects1[aspect]] = 3
        except:
            b[aspects1[aspect]] = 0
    for i in range(len(b)):
        if b[i]>0:
            b[i]=1
        elif b[i]<0:
            b[i] = -1
        else:
            b[i]=0
    output.append(b)
print(output)

{'Service': '5', 'Cleanliness': '5', 'Overall': '5.0', 'Value': '5', 'Sleep Quality': '5', 'Rooms': '5', 'Location': '5'}
{'Service': '4', 'Cleanliness': '4', 'Overall': '4.0', 'Value': '4', 'Sleep Quality': '4', 'Rooms': '4', 'Location': '5'}
{'Service': '5', 'Cleanliness': '5', 'Overall': '4.0', 'Value': '3', 'Sleep Quality': '3', 'Rooms': '5', 'Location': '5'}
{'Service': '5', 'Cleanliness': '5', 'Overall': '5.0', 'Value': '4', 'Sleep Quality': '5', 'Rooms': '5', 'Location': '5'}
{'Service': '5', 'Cleanliness': '5', 'Overall': '5.0', 'Value': '4', 'Sleep Quality': '5', 'Rooms': '5', 'Location': '5'}
{'Service': '5', 'Cleanliness': '5', 'Overall': '4.0', 'Value': '4', 'Sleep Quality': '5', 'Rooms': '5', 'Location': '5'}
{'Cleanliness': '5', 'Sleep Quality': '5', 'Overall': '4.0', 'Value': '3', 'Service': '4'}
{'Cleanliness': '4', 'Sleep Quality': '3', 'Overall': '4.0', 'Value': '4', 'Service': '5'}
{'Cleanliness': '4', 'Sleep Quality': '4', 'Overall': '4.0', 'Value': '4', 'Service': 

In [11]:
check_accuracy(predicted,output)

[0.7856135401974612,
 0.6829896907216495,
 0.7091388400702988,
 0.6677148846960168,
 0.7425044091710759]